# K Means Algrithm

## Introduction

## Load and Display data

In [ ]:
from sklearn.datasets import load_iris
import matplotlib.pylab as plt
import numpy as np
iris = load_iris()
data_iris=iris.data

plt.figure()
plt.subplots()
plt.scatter(data_iris[:,0][iris.target==0],data_iris[:,1][iris.target==0],label=iris.target_names[0])
plt.scatter(data_iris[:,0][iris.target==1],data_iris[:,1][iris.target==1],label=iris.target_names[1])
plt.scatter(data_iris[:,0][iris.target==2],data_iris[:,1][iris.target==2],label=iris.target_names[2])
plt.xlabel(iris.feature_names[0])
plt.ylabel(iris.feature_names[3])
plt.legend()

## Define Functions

In [ ]:
class K_means:
    def __init__(self,data,k=3,dis_type='euclidean',seed=4,res=1e-10,n_loop=1000):
        self.data = data
        self.k = k
        self.dis_type = dis_type
        self.res = res
        self.n_loop = n_loop
        self.seed=seed
    def initial_k(self,data,k,seed):
        '''
        usage: k_points_initial = initial_k(data=data_iris,k=3);
        data: input data, nd array, (N,D);
        k: cluster number;
        dis_type: distance type.
        '''
        import numpy as np
        dim = data.shape[1]
        k_initial = np.zeros((k,dim))
        for i_dim in np.arange(dim):
            #set seeds
            if seed!=0:
                np.random.seed(seed+i_dim)
            k_initial[:,i_dim] = np.random.uniform(np.amin(data[:,i_dim]),np.amax(data[:,i_dim]),k)
        return k_initial
    
    def distance_kmean(self,data,k_points,dis_type):
        '''
        data: input data;
        k_points: k number of points;
        usage: distance = distance_kmean(data=data_iris,k_points=k_points_initial)
        '''
        import numpy as np
        import numpy.linalg as LA
        n_sample = data.shape[0]
        k = k_points.shape[0]
        distance = np.zeros((n_sample,k))
        for i_k in np.arange(k):
            i_k_point_datashape = np.repeat(np.expand_dims(k_points[i_k,:],axis=0),n_sample,axis=0)
            distance_array= data-i_k_point_datashape
            distance_i_k = LA.norm(distance_array,2,axis=1)
            distance[:,i_k] = distance_i_k
        return distance
    
    def centroids_cal(self,data,distance,k_points_initial):
        '''
        data: input data;
        distance: distance calculated;
        k_points_initial:
        k_points_update = centroids_cal(data=data_iris,distance=distance,k_points_initial=k_points_initial)
        '''
        import numpy as np
        new_group_labels = np.argmin(distance,axis=1)
        k = k_points_initial.shape[0]
        k_points_new = np.zeros(k_points_initial.shape)
        for i_k in np.arange(k):
            index = np.array(new_group_labels)==i_k
            if len(index)>0:
                data_i_k = data[index,:]
                k_points_new[i_k,:] = np.mean(data_i_k,axis=0)
            else:
                k_points_new[i_k,:] = k_points_initial[i_k,:]
        return k_points_new
    # def sort_labels(self,targets,labels):
        # n_label_max = len(np.unique(labels))
        # n_label = n_label_max-1
        # for label in np.unique(labels):
        #     n_label+=1
        #     labels[labels==label] = n_label
        # labels = labels-n_label_max
    def k_means(self,data,k,dis_type,seed,res=1e-10,n_loop=1000):
        '''

        '''
        import numpy as np
        import numpy.linalg as LA
        k_points_old = self.initial_k(data=data,k=k,seed=seed)
        distance = self.distance_kmean(data=data,k_points=k_points_old,dis_type=self.dis_type)
        #
        k_points_stored = list([])
        k_points_stored.append(k_points_old)
        n_cal = 0
        distance_move = res+1
        while (distance_move >res) and (n_cal<n_loop):
            n_cal+=1
            distance = self.distance_kmean(data=data,k_points=k_points_old,dis_type=self.dis_type)
            k_points_update = self.centroids_cal(data=data,distance=distance,k_points_initial=k_points_old)
            distance_move = np.sum(LA.norm(k_points_old-k_points_update,2,axis=1))
            k_points_old = k_points_update
            k_points_stored.append(k_points_old)
        k_points_stored = np.array(k_points_stored)
        distance = self.distance_kmean(data=data,k_points=k_points_old,dis_type=self.dis_type)
        # assigin right ordered labels
        labels = np.argmin(distance,axis=1)
        # output
        return labels, k_points_stored


In [ ]:
# function usage

## Fitting

In [ ]:
data_iris=iris.data[:,[1,2,3]]
km = K_means(data=data_iris)
tol = 0.8
accuracy =0
# res_index = 150
seed = 3
while accuracy < tol:
    seed+=1
    labels,k_points_stored = km.k_means(data=data_iris,k=3,dis_type='euclidean',seed=seed,res=1e-3,n_loop=1500)
    index_wrong = labels-iris.target
    
    res_index = np.sum(np.abs(index_wrong))
    accuracy = np.array(len(index_wrong[index_wrong==0])/len(labels)).round(decimals=2)
print('accuracy: ',accuracy)
print(seed)

## Display results

### Display confusion matrix and accuracy

In [ ]:
from sklearn.metrics import confusion_matrix
confu = confusion_matrix(iris.target,labels)
plt.imshow(confu,cmap=plt.cm.Blues)
plt.colorbar()
plt.xticks(np.arange(3),iris.target_names)
plt.yticks(np.arange(3),iris.target_names)
plt.xlabel('Prediction')
plt.ylabel('Truth')
for row in np.arange(3):
    for col in np.arange(3):
        plt.text(row,col,confu.T[row,col])
plt.title('confusion matrix (seed= '+str(seed)+')')
plt.savefig('./image/k_mean_iris_confusion_matrix.png',dpi=300)
plt.show()

### Display centriod paths

In [ ]:
import matplotlib.pylab as plt
import numpy as np
colors = ['green','orange','blue']
plt.figure()
plt.subplots()
plt.scatter(data_iris[:,0][iris.target==0],data_iris[:,1][iris.target==0],alpha=0.3,marker='.'
            ,sizes=15*np.ones(data_iris.shape[0]),color=colors[0])#label=iris.target_names[0]
plt.scatter(data_iris[:,0][iris.target==1],data_iris[:,1][iris.target==1],alpha=0.3,marker='.'
            ,sizes=15*np.ones(data_iris.shape[0]),color=colors[1])#,label=iris.target_names[1]
plt.scatter(data_iris[:,0][iris.target==2],data_iris[:,1][iris.target==2],alpha=0.3,marker='.'
            ,sizes=15*np.ones(data_iris.shape[0]),color=colors[2])#,label=iris.target_names[2]
plt.scatter(k_points_stored[0,:,0],k_points_stored[0,:,1],alpha=0.9,marker='*'
            ,sizes=200*np.ones(data_iris.shape[0]),color='red')#,label=iris.target_names[2]
#plot wrong prediction

plt.scatter(data_iris[:,0][index_wrong!=0],data_iris[:,1][index_wrong!=0],alpha=0.3,marker='x'
            ,sizes=15*np.ones(data_iris.shape[0]),color='black')#,label=iris.target_names[2]
# plot k path
for i_kmean in np.arange(k_points_stored.shape[1]):
    plt.plot(k_points_stored[:,i_kmean,0],k_points_stored[:,i_kmean,1],'--',marker='s',
                color=colors[i_kmean%3],label='path: '+str(i_kmean),alpha=0.5)
plt.xlabel(iris.feature_names[0])
plt.ylabel(iris.feature_names[3])
plt.legend()
plt.title('k means centriod moving paths: iris data'+', accuracy='+str(accuracy))
plt.savefig('./image/k_mean_iris_centriod_paths.png',dpi=300)
plt.show()